In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [54]:
dev= pd.read_csv('devices.csv')
tr1= pd.read_csv('transactions_1.csv')
tr2= pd.read_csv('transactions_2.csv')
tr3= pd.read_csv('transactions_3.csv')
users = pd.read_csv('users.csv')
notif = pd.read_csv('notifications.csv')
trans= pd.concat([tr1, tr2, tr3])#all transactions in one dataframe

#users table

In [55]:
users['brands']=dev['brand']#h sthlh brands apo ta devices is added to users table
#ginontai ta unknown stoixeia tyxaia Android, Apple
import random
def change_unknown_device(brand):
  if  brand== 'Unknown':
    return random.sample(['Android', 'Apple'], 1)[0]
  else:
    return brand
users['brands']=users['brands'].apply(change_unknown_device)


In [56]:
users['age']=2019 - users['birth_year']#sthlh age anti gia birth_year

In [57]:
users['plan']=users['plan'].apply(lambda x: 'PAID' if x!="STANDARD" else x)#h sthlh pairnei pleon standard h paid stoixeia

In [58]:
#handling dates in days
users.created_date=pd.to_datetime(users.created_date)
#users['today']=np.datetime64('2019-06-01')
users['accound_duration'] = (np.datetime64('2019-06-01') - users['created_date']).dt.days

In [59]:
#users['month']=pd.DatetimeIndex(users['created_date']).month
#users['year']=pd.DatetimeIndex(users['created_date']).year
#users['duration']=users.apply(lambda x: 18-x['month'] if x['year']==2018 else 6-x['month'], axis=1)

#transactions table
-------------------

In [60]:
trans=trans[trans['transactions_state']=='COMPLETED']#kratame mono ta completed transactions

In [61]:
number_of_trans=pd.DataFrame(trans.user_id.value_counts())#amount of transactions per user
number_of_trans.rename(columns={'user_id':'total_amount_of_transactions'},inplace=True)
users=pd.merge(users , number_of_trans, how='left', left_on='user_id', right_index=True)#h sthlg amount_of_transactions is joined sto users

In [62]:
trans_out=trans[trans['direction']=='OUTBOUND']
trans_in=trans[trans['direction']=='INBOUND']

In [63]:
usd_per_user_out = pd.pivot_table(data=trans_out,aggfunc=['count','min','max','mean','sum'], index='user_id',  values='amount_usd')
usd_per_user_out.columns=['amount_of_transactions_out', 'min_usd_out', 'max_usd_out', 'mean_usd_out', 'sum_usd_out']
users=pd.merge(users , usd_per_user_out, how='left', left_on='user_id', right_index=True)

In [64]:
usd_per_user_in = pd.pivot_table(data=trans_in,aggfunc=['count','min','max','mean','sum'], index='user_id',  values='amount_usd')
usd_per_user_in.columns=['amount_of_transactions_in', 'min_usd_in', 'max_usd_in', 'mean_usd_in', 'sum_usd_in']
users=pd.merge(users , usd_per_user_in, how='left', left_on='user_id', right_index=True)

In [65]:
#pd.crosstab(trans.transactions_type, trans.ea_merchant_country)

In [66]:
#pd.crosstab(trans.transactions_type, trans.ea_cardholderpresence)

In [67]:
trans_transactions_type_out = pd.crosstab(trans_out['user_id'], columns=trans_out['transactions_type'])
trans_transactions_type_in = pd.crosstab(trans_in['user_id'], columns=trans_in['transactions_type'])
trans_transactions_type_out.columns=['count_type_atm', 'count_type_card_payment','count_type_exchange','count_type_fee', 'count_type_tax', 'count_type_transfer_out']
trans_transactions_type_in.columns=['count_type_card_refund', 'count_type_cashback','count_type_refund','count_type_topup','count_type_transfer_in']

In [68]:
users=pd.merge(users , trans_transactions_type_out, how='left', left_on='user_id', right_index=True)

In [69]:
users=pd.merge(users , trans_transactions_type_in, how='left', left_on='user_id', right_index=True)

In [70]:
#trans['month']=pd.DatetimeIndex(trans['created_date']).month#created month and year as columns
#trans['year']=pd.DatetimeIndex(trans['created_date']).year


In [71]:
trans_sorted=trans.sort_values(['created_date'],ascending=False)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
lasttrans = trans_sorted.drop_duplicates('user_id').sort_index()
lasttrans=lasttrans[['user_id','created_date']]

In [72]:
#handling dates in days
lasttrans['created_date']=pd.to_datetime(lasttrans['created_date'])
#lasttrans['today']=np.datetime64('2019-06-01')
lasttrans['last_transaction'] = (np.datetime64('2019-06-01') - lasttrans['created_date']).dt.days

In [73]:
#lasttrans['month']=pd.DatetimeIndex(lasttrans['created_date']).month
#lasttrans['year']=pd.DatetimeIndex(lasttrans['created_date']).year
#lasttrans['months_from_last_transaction']=lasttrans.apply(lambda x: 17-x['month'] if x['year']==2018 else 5-x['month'], axis=1)

In [74]:
users=pd.merge(users , lasttrans, how='left', left_on='user_id', right_on='user_id')

In [75]:
trans_sorted_asc=trans.sort_values(['created_date'],ascending=True)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
firsttrans = trans_sorted_asc.drop_duplicates('user_id').sort_index()
firsttrans=firsttrans[['user_id','created_date']]

In [76]:
#handling dates in days
firsttrans['created_date']=pd.to_datetime(firsttrans['created_date'])
#firsttrans['today']=np.datetime64('2019-06-01')
firsttrans['first_transaction'] = (np.datetime64('2019-06-01')-firsttrans['created_date']).dt.days

In [77]:
#lasttrans['month']=pd.DatetimeIndex(lasttrans['created_date']).month
#lasttrans['year']=pd.DatetimeIndex(lasttrans['created_date']).year
#lasttrans['months_from_first_transaction']=lasttrans.apply(lambda x: 18-x['month'] if x['year']==2018 else 6-x['month'], axis=1)

In [78]:
users=pd.merge(users , firsttrans, how='left', left_on='user_id', right_on='user_id')

In [79]:
users.sample(4)

,user_id,birth_year,country,city,created_date_x,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,...,count_type_transfer_out,count_type_card_refund,count_type_cashback,count_type_refund,count_type_topup,count_type_transfer_in,created_date_y,last_transaction,created_date,first_transaction
11710,user_11710,1974,GB,Linlithgow,2018-09-21 23:08:50.564972,0,STANDARD,1.0,1.0,19,...,NaN,0.0,0.0,0.0,1.0,0.0,2018-09-21 23:14:56.100972,252.0,2018-09-21 23:14:56.100972,252.0
7281,user_7281,1999,CZ,České Meziříčí,2018-08-31 18:26:47.995709,1,STANDARD,1.0,0.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN
4772,user_4772,1999,IE,Mountmellick,2018-08-25 10:56:08.124710,0,STANDARD,1.0,1.0,0,...,8.0,0.0,0.0,0.0,0.0,4.0,2019-03-26 07:27:40.520710,66.0,2018-09-12 02:56:40.637710,261.0
4237,user_4237,1995,RO,Galati,2018-10-18 18:16:32.747989,0,STANDARD,NaN,NaN,43,...,10.0,1.0,0.0,0.0,4.0,7.0,2019-05-11 20:07:35.542989,20.0,2018-10-18 18:22:54.794989,225.0


In [80]:
users['engagement_duration']=users['first_transaction'] - users['last_transaction'] + 1

In [81]:
users.columns

Index(['user_id', 'birth_year', 'country', 'city', 'created_date_x',
       'user_settings_crypto_unlocked', 'plan',
       'attributes_notifications_marketing_push',
       'attributes_notifications_marketing_email', 'num_contacts',
       'num_referrals', 'num_successful_referrals', 'brands', 'age',
       'accound_duration', 'total_amount_of_transactions',
       'amount_of_transactions_out', 'min_usd_out', 'max_usd_out',
       'mean_usd_out', 'sum_usd_out', 'amount_of_transactions_in',
       'min_usd_in', 'max_usd_in', 'mean_usd_in', 'sum_usd_in',
       'count_type_atm', 'count_type_card_payment', 'count_type_exchange',
       'count_type_fee', 'count_type_tax', 'count_type_transfer_out',
       'count_type_card_refund', 'count_type_cashback', 'count_type_refund',
       'count_type_topup', 'count_type_transfer_in', 'created_date_y',
       'last_transaction', 'created_date', 'first_transaction',
       'engagement_duration'],
      dtype='object')

In [82]:
users['days_passed_for_first_transaction']= users['accound_duration']- users['first_transaction'] 

In [83]:
users.sample()

,user_id,birth_year,country,city,created_date_x,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,...,count_type_cashback,count_type_refund,count_type_topup,count_type_transfer_in,created_date_y,last_transaction,created_date,first_transaction,engagement_duration,days_passed_for_first_transaction
14917,user_14917,1982,IE,Arklow,2018-08-27 13:20:35.629112,0,STANDARD,1.0,1.0,134,...,NaN,NaN,NaN,NaN,2019-03-26 07:20:34.375112,66.0,2019-03-04 11:05:57.443112,88.0,23.0,189.0


In [84]:
users['transactions_frequency']=users['total_amount_of_transactions'] / users['engagement_duration']

#Notifications
________________________________________________________


In [85]:
notif_sent=notif[notif['status']=='SENT']


In [86]:
amount_notif_sent=pd.crosstab(notif_sent['user_id'], columns=[notif_sent['channel'],notif_sent['reason']])
amount_notif_sent.columns=['_'.join(col).lower() for col in amount_notif_sent.columns.values]

In [87]:
users=pd.merge(users, amount_notif_sent, how='left', left_on='user_id', right_index=True)


###notifications table analysis under process

In [88]:
#pd.crosstab(notif['reason'], columns=notif['status'])

In [89]:
#pd.crosstab(notif['user_id'], columns=[notif['channel'], notif['status'], notif['reason']]).head(5)

In [90]:
#notif[notif['user_id']=='user_5734'].sort_values('created_date')#Z

In [91]:
#users[users['user_id']=='user_12544']

In [92]:
#=pd.crosstab(notif['user_id'], columns= notif['reason'])
#tepm2.sample(5)

In [93]:
#=pd.merge(users, tepm2, left_on='user_id', right_index=True )

In [94]:
#=temp[['plan','BLACK_FRIDAY','BLUE_TUESDAY','ENGAGEMENT_SPLIT_BILL_RESTAURANT','ENGAGEMENT_SPLIT_BILL_RESTAURANT','INVEST_IN_GOLD','JOINING_ANNIVERSARY','LOST_CARD_ORDER','MADE_MONEY_REQUEST_NOT_SPLIT_BILL','METAL_RESERVE_PLAN','NO_INITIAL_CARD_ORDER','NO_INITIAL_CARD_USE','ONBOARDING_TIPS_ACTIVATED_USERS','PROMO','PROMO_CARD_ORDER','REENGAGEMENT_ACTIVE_FUNDS','SILVER_ENGAGEMENT_FEES_SAVED','SILVER_ENGAGEMENT_INACTIVE_CARD','WELCOME_BACK']]

In [95]:
#temp[temp['plan']=='PAID'].sample(50)

#Dropping and encoding users table

In [96]:
#dropping columns we don't need
data=users.drop(columns=['user_id','birth_year','city','created_date_x','attributes_notifications_marketing_push','attributes_notifications_marketing_email','num_referrals','num_successful_referrals','created_date_y','created_date'])


#'year_x','month_x','month_y','year_y','month','year'

In [97]:
data.columns

Index(['country', 'user_settings_crypto_unlocked', 'plan', 'num_contacts',
       'brands', 'age', 'accound_duration', 'total_amount_of_transactions',
       'amount_of_transactions_out', 'min_usd_out', 'max_usd_out',
       'mean_usd_out', 'sum_usd_out', 'amount_of_transactions_in',
       'min_usd_in', 'max_usd_in', 'mean_usd_in', 'sum_usd_in',
       'count_type_atm', 'count_type_card_payment', 'count_type_exchange',
       'count_type_fee', 'count_type_tax', 'count_type_transfer_out',
       'count_type_card_refund', 'count_type_cashback', 'count_type_refund',
       'count_type_topup', 'count_type_transfer_in', 'last_transaction',
       'first_transaction', 'engagement_duration',
       'days_passed_for_first_transaction', 'transactions_frequency',
       'email_engagement_split_bill_restaurant', 'email_invest_in_gold',
       'email_lost_card_order', 'email_made_money_request_not_split_bill',
       'email_no_initial_card_order', 'email_no_initial_card_use',
       'email_onboar

In [98]:
data.isnull().sum()# ta nan tha ginoyn 0, afoy einai eite counts eite sum klp,kai 
datatest=data.fillna(0)#dhmioyrgeitai o datatest gia xrhsh se modela

In [99]:
#column plan  encoding sto dataset
plan_dict = {'STANDARD': 0, 'PAID': 1}
datatest['plan'] = datatest.plan.map(plan_dict)



In [100]:
datatest.drop('country', axis=1, inplace=True)
#gia thn wra petaw ta countries mexri na doyme pws ua ta xeiristoume

In [101]:
#column brands encoding sto dataset
datatest=pd.get_dummies(datatest)

In [105]:
#dhmioyrgoyntai oi pinakes pou exw anevasei
datatest.to_csv('days_for_testing.csv',index=False, sep=';')
data.to_csv('days_from_users.csv',index=False, sep=';')
users.to_csv('days_users_feat.csv',index=False, sep=';')

In [103]:
datatest[datatest['transactions_frequency']==np.inf]

,user_settings_crypto_unlocked,plan,num_contacts,age,accound_duration,total_amount_of_transactions,amount_of_transactions_out,min_usd_out,max_usd_out,mean_usd_out,...,push_joining_anniversary,push_lost_card_order,push_promo,push_reengagement_active_funds,push_silver_engagement_fees_saved,push_silver_engagement_inactive_card,sms_metal_reserve_plan,sms_reengagement_active_funds,brands_Android,brands_Apple


In [104]:
#dhmioyrgia X, y pinakwn gia na etoimasia toy train_test_split
#X = us_1_en.drop(columns=['plan']).values
#X_columns = us_1_en.drop(columns=['plan']).columns
#y = us_1_en['plan'].values